# ERS NIRCam Demo with Eureka!
## Authors: Megan Mansfield and Eva-Maria Ahrer
### Hello! This notebook shows a full demonstration reducing and analyzing the NIRCam/F322W2 transit of WASP-39b from the JWST Transiting Exoplanet Early Release Science Program (ERS 1366). Following this pipeline, you should be able to reproduce the NIRCam spectrum presented in the main text of Ahrer et al. (2022). 
### This data reduction uses the [Eureka!](https://github.com/kevin218/Eureka) pipeline. Before running this Jupyter Notebook, please follow the steps in "README.md" to set up an environment for this demonstration and install Eureka. Note that this demonstration is using a fixed version of Eureka! to ensure that no future updates break the example here, so even if you have the current version of Eureka! installed on your own machine you'll need to follow the set-up steps to ensure this demo works properly. For more information on Eureka! see its documentation or refer to Bell et al. (2022).

## First, download the data set.
### Data can be downloaded from MAST at: (DOI? Specify which file extensions/stage?)

## Next, let's import packages!

In [3]:
import sys
sys.path.append('./')
sys.path.insert(0,'./')
import eureka.S1_detector_processing.s1_process as s1
import eureka.S2_calibrations.s2_calibrate as s2
import eureka.S3_data_reduction.s3_reduce as s3
import eureka.S4_generate_lightcurves.s4_genLC as s4
import eureka.S5_lightcurve_fitting.s5_fit as s5
import eureka.S6_planet_spectra.s6_spectra as s6

eventlabel = 'nircam_wfss_template'
ecf_path = './ecf/'

## Eureka! is divided into six "Stages". Each Stage has a corresponding Eureka Control File, or .ecf file. These files are named "S2_eventlabel.ecf" where the first half of the name (S2, S3, etc.) refers to the Stage that .ecf file interacts with, and the second half refers to the "eventlabel" keyword we defined above. See the Eureka! documentation for a full description of each stage and all keywords in the .ecf files. Here we'll give a very brief summary of each Stage and discuss a few important keywords for reproducing the paper results.

## Stage 1: Correcting detector-level effects and fitting the up-the-ramp slope.